['id','click','hour','banner_pos','device_id','device_ip','device_model','device_conn_type','C14','C17','C20','C21'], and add cols ['pub_id','pub_domain','pub_category','device_id_count','device_ip_count','user_count','smooth_user_hour_count','user_click_histroy'].

Stats:
- 'small' with device_* cols: Private=0.3998675, Public=0.4020308
- 'small' without device_* cols: Private=0.4024273, Public=0.4045937


TODO:
- add features: device_id, device_ip, and device_model
- count features: device_id, device_ip, user
- expedite data generation. An iteration of a model is too slow.

In [1]:
import xlearn as xl

from tools.ffm_tools import make_train_validate_data, df_to_ffm, write_submission
from models.base import create_user, site_app_split

## Expedite data generation

In [2]:
import pandas as pd

categorical_features = ['banner_pos', 'platform_id', 'platform_domain', 'platform_category',
                    'user', 'device_conn_type', 'C14','C17','C20','C21']
#  add device_id, device_ip, and device_model.
#categorical_features = ['banner_pos', 'platform_id', 'platform_domain', 'platform_category',
#                        'device_id', 'device_ip', 'device_model',
#                    'user', 'device_conn_type', 'C14','C17','C20','C21']

train_size='tiny'

df = pd.read_csv(f'./data/train_{train_size}.csv')
df.hour = pd.to_datetime(df.hour, format="%y%m%d%H")
df = create_user(df)

In [3]:
from tools.cv_tools import train_test_split

test_day = 30
df_train, df_validate = train_test_split(df, None, test_day)
df_train_site, df_train_app = site_app_split(df_train)
df_validate_site, df_validate_app = site_app_split(df_validate)

In [4]:
df_train_site.shape

(590, 23)

In [5]:
from tools.ffm_tools import make_feature_dict, encode_features

feature_dict_site = make_feature_dict(df_train_site, categorical_features)

In [6]:
encoded = encode_features(df_train_site, feature_dict_site, categorical_features)

In [8]:
encoded[categorical_features].head()

,banner_pos,platform_id,platform_domain,platform_category,user,device_conn_type,C14,C17,C20,C21
0,0,2,152,274,283,869,871,1095,1214,1259
1,0,2,152,274,284,869,872,1096,1215,1260
2,0,3,153,275,285,869,873,1097,1214,1259
3,0,4,154,276,286,869,874,1098,1214,1260
4,1,5,155,276,287,869,875,1099,1216,1259


In [7]:
raise

RuntimeError: No active exception to reraise

## Without count features

### Part 0: prepare train data.

Run `~/code/xlearn/build/xlearn_train ./ffm-data/train_{site,app}_{train_size}.ffm -s 2 -v ./ffm-data/validate_{site,app}_{train_size}.ffm`.

Generates `./ffm-data/train_{site,app}_{train_size}.ffm.model`.

Early-stopping is enabled by default when a validation set is provided.

Stats:
- 'small', with categorical_features = ['banner_pos', 'platform_id', 'platform_domain', 'platform_category', 'user', 'device_conn_type', 'C14','C17','C20','C21'] =>
- 'small', with categorical_features = ['banner_pos', 'platform_id', 'platform_domain', 'platform_category',
    'device_id', 'device_ip', 'device_model',
    'user', 'device_conn_type', 'C14','C17','C20','C21'] => 09m15s, best losses: site-0.447526, app-0.343323.

In [ ]:
import pandas as pd

categorical_features = ['banner_pos', 'platform_id', 'platform_domain', 'platform_category',
                    'user', 'device_conn_type', 'C14','C17','C20','C21']
#  add device_id, device_ip, and device_model.
#categorical_features = ['banner_pos', 'platform_id', 'platform_domain', 'platform_category',
#                        'device_id', 'device_ip', 'device_model',
#                    'user', 'device_conn_type', 'C14','C17','C20','C21']

train_size='small'

df = pd.read_csv(f'./data/train_{train_size}.csv')
df.hour = pd.to_datetime(df.hour, format="%y%m%d%H")
df = create_user(df)

In [ ]:
%%time
train_site, validate_site, feature_dict_site,\
train_app, validate_app, feature_dict_app = make_train_validate_data(df, categorical_features, train_size)

In [ ]:
run_training_site = f"""
~/code/xlearn/build/xlearn_train
./ffm-data/train_site_{train_size}.ffm
-s 2 -v ./ffm-data/validate_site_{train_size}.ffm
"""
run_training_site.replace('\n',' ').strip()

In [ ]:
run_training_app = f"""
~/code/xlearn/build/xlearn_train
./ffm-data/train_app_{train_size}.ffm
-s 2 -v ./ffm-data/validate_app_{train_size}.ffm
"""
run_training_app.replace('\n',' ').strip()

In [ ]:
raise "Train models."

### Part 1: prepare test data.

Generates `./ffm-data/test_{site,app}_{test_size}.ffm.out`.

Stats:
- 'full', with categorical_features = ['banner_pos', 'platform_id', 'platform_domain', 'platform_category', 'user', 'device_conn_type', 'C14','C17','C20','C21']  --> site = 27m; app = 14m.
- 'small', with categorical_features = ['banner_pos', 'platform_id', 'platform_domain', 'platform_category', 'user', 'device_conn_type', 'C14','C17','C20','C21']  --> site = 06m; app = 4m.
- 'full', with categorical_features = ['banner_pos', 'platform_id', 'platform_domain', 'platform_category',
    'device_id', 'device_ip', 'device_model',
    'user', 'device_conn_type', 'C14','C17','C20','C21'] => site = 25m39s; app = 15m45s.

In [ ]:
test_size='full'

if test_size == 'full':
    df_test = pd.read_csv(f'./data/test.csv', dtype={'id': 'uint64'})
else:
    df_test = pd.read_csv(f'./data/test_{test_size}.csv', dtype={'id': 'uint64'})
df_test = create_user(df_test)

df_test_site, df_test_app = site_app_split(df_test)
del df_test
df_test_site.shape, df_test_app.shape

In [ ]:
%%time
ffm_data_test_site = f'./ffm-data/test_site_{test_size}.ffm'
    
df_to_ffm(df_test_site, categorical_features, 
          ffm_data_test_site, 'test', feature_dict_site)

In [ ]:
%%time
ffm_data_test_app = f'./ffm-data/test_app_{test_size}.ffm'

df_to_ffm(df_test_app, categorical_features, 
          ffm_data_test_app, 'test', feature_dict_app)

In [ ]:
# result of 
# ~/code/xlearn/build/xlearn_predict ./ffm-data/test_{site,app}_tiny.ffm ./ffm-data/train_{site,app}_tiny.ffm.model --sigmoid
# => success. Generates test_{site,app}_tiny.ffm.out

In [ ]:
run_predict_site = f"""
~/code/xlearn/build/xlearn_predict
./ffm-data/test_site_{test_size}.ffm
./ffm-data/train_site_{train_size}.ffm.model --sigmoid
"""
run_predict_site.replace('\n',' ').strip()

In [ ]:
run_predict_app = f"""
~/code/xlearn/build/xlearn_predict
./ffm-data/test_app_{test_size}.ffm
./ffm-data/train_app_{train_size}.ffm.model --sigmoid
"""
run_predict_app.replace('\n',' ').strip()

In [ ]:
raise "run prediction"

### Part 2: write submission.csv

In [ ]:
prediction_site = f'./ffm-data/test_site_{test_size}.ffm'
prediction_site_out = prediction_site + '.out'
prediction_site_id = prediction_site + '.id'
prediction_app = f'./ffm-data/test_app_{test_size}.ffm'
prediction_app_out = prediction_app + '.out'
prediction_app_id = prediction_app + '.id'

write_submission(prediction_site_out, prediction_site_id, prediction_app_out, prediction_app_id, 'submission-.csv')

### Part 2 (obsolete): train model and predict.
Do this in shell.

In [ ]:
def run_ffm(train_set, validate_set, test_set, model_out, predict_out):
    ffm_model = xl.create_ffm()
    ffm_model.setTrain(train_set)
    ffm_model.setValidate(validate_set)
    
    param = {'task':'binary', 'lr':0.2, 'lambda':0.002}
    ffm_model.fit(param, model_out)
    
    ffm_model.setSigmoid()
    ffm_model.setTest(test_set)
    ffm_model.predict(model_out, predict_out)
    print(f'Prediction written to {predict_out}')

In [ ]:
#TODO: move this to tools/kaggle_tools
def validate_submission(submission_csv, ids):
    """
    Ensure that the submission file is in the right format, 
    and that it contains required id's.
    """
    pass

In [ ]:
train_size = 'small'
test_size = 'small'

In [ ]:
train_site = f'./ffm-data/train_site_{train_size}.ffm'
validate_site = f'./ffm-data/validate_site_{train_size}.ffm'
test_site = f'./ffm-data/test_site_{test_size}.ffm'
model_site_out = './xlearn-out/ffm-site.out'
predict_site_out = './xlearn-out/out-site.txt'

In [ ]:
%%time
# TODO: kernel dies when the test 
run_ffm(train_site, validate_site, test_site, model_site_out, predict_site_out)

In [ ]:
train_app = f'./ffm-data/train_app_{train_size}.ffm'
validate_app = f'./ffm-data/validate_app_{train_size}.ffm'
test_app = f'./ffm-data/test_app_{test_size}.ffm'
model_app_out = './xlearn-out/ffm-app.out'
predict_app_out = './xlearn-out/out-app.txt'

run_ffm(train_app, validate_app, test_app, model_app_out, predict_app_out)

In [ ]:
prediction_id_site = f'./ffm-data/test_site_{test_size}.ffm.id'
prediction_id_app = f'./ffm-data/test_app_{test_size}.ffm.id'
write_submission(predict_site_out, prediction_id_site, predict_app_out, prediction_id_app, 'submission-.csv')